# Production Planning Problem - Extended
This is an extended version of the production planning problem.

Suppose that we are responsible for scheduling the monthly production plan for multiple products in a year. Here are some assumptions:
- The demand of the product, unit production cost, production capacity and inventory capacity in each month are given.
- Inventory holding costs are assessed at the end of each month.
- Holding cost per unit is invariant from month to month.
- There are 500, 700 units of inventory available at the beginning of the first month for the two products resp.
- Production should always meet the demand and should not exceed the stock threshold

### Mathematical Formulation
Let us derive the formulation from the problem description.

##### Sets:
- **t**: the set of time periods (months)

##### Parameters:
- **h**: unit holding cost
- **p**: production capacity per month
- **I_0**: the initial inventory
- **c_t**ₜ: unit production cost in month t
- **d_t**: demand of month t
- **i**: inventory capacity per month
- **s**: stock threshold per month

##### Variables:
- **X_t**: amount produced in month t
- **I_t**: inventory at the end of period t

##### Objective:
**Minimize: (h x I_t + c_t x X_t)**

###### Constraints
- Inventory: I_{t-1} + X_t - d_t = I_t
- Production: I_{t-1} + X_t <= d_t + (d_t * s)
- Production Capacity: X_t <= p
- Inventory Capacity: I_t <= i
- Low Bound: X_t, I_t >= 0

In [1]:
# Import PuLP modeler functions
from pulp import *
import pandas as pd

In [2]:
input_data = pd.read_excel("D:\Projects\Marico\InventoryOptimization\Extended Production Planning\Product_Wise_Demand.xlsx")
input_parameters = pd.read_excel("D:\Projects\Marico\InventoryOptimization\Extended Production Planning\Product_Wise_Parameters.xlsx")

In [3]:
input_data

,product,period,demand,production_cost,production_capacity,inventory_capacity
0,Coke,1,3300,150,6400,7360
1,Coke,2,3300,150,6400,7360
2,Coke,3,4500,140,6800,7820
3,Coke,4,6100,140,6800,7820
4,Coke,5,5100,150,6400,7360
5,Coke,6,9000,135,7000,8050
6,Coke,7,9700,135,7000,8050
7,Coke,8,8200,135,7000,8050
8,Coke,9,7900,135,6800,7820
9,Coke,10,5700,140,6800,7820


In [4]:
input_data['product'].unique()

array(['Coke', 'Juice'], dtype=object)

In [5]:
input_parameters

,product,attribute,value
0,Coke,holding_cost,8.00
1,Coke,initial_inventory,500.00
2,Coke,stock_threshold,0.15
3,Juice,holding_cost,7.00
4,Juice,initial_inventory,700.00
5,Juice,stock_threshold,0.10


In [6]:
df_holding_costs = pd.DataFrame({'Product': [], 'Inventory_Variable': [], 'Unit_Holding_Cost': []})
df_production_costs = pd.DataFrame({'Production_Variable': [], 'Unit_Production_Cost': []})
df_model_parameters = pd.DataFrame({'Model_Name': [], 'Model_Objective': [], 'Model_Status': [], 'Objective_Value': []})
df_costs = pd.DataFrame({'Product': [], 'Inventory_Variable': [], 'Unit_Holding_Cost': [], 
                            'Production_Variable': [], 'Unit_Production_Cost': []})

for product in input_data['product'].unique():
    
    print('************************************')
    print('Product = ', product)
    
    # Create a dictionary from the available data
    input_df_dict = {}
    input_df_dict['input_data'] = input_data[input_data['product'] == product].reset_index()
    input_df_dict['parameters'] = input_parameters[input_parameters['product'] == product].reset_index()
    
    # As the parameters are few, a dictionary is created separately
    input_param_dict = input_df_dict['parameters'].set_index('attribute')['value'].to_dict()
    
    # Create the model variable to contain the problem data
    model = LpProblem(name='Production_Planning_for_'+product, sense=LpMinimize)

    # Decision variables
    production_variables = LpVariable.dicts(name='X', indexs=input_df_dict['input_data'].index,
                                            lowBound=0, cat=LpContinuous)
    # i.e, {0: X_0, 1: X_1, 2: X_2, 3: X_3, 4: X_4, 5: X_5, 6: X_6, 7: X_7, 8: X_8, 9: X_9, 10: X_10, 11: X_11}

    inventory_variables = LpVariable.dicts(name='I', indexs=input_df_dict['input_data'].index,
                                           lowBound=0, cat=LpContinuous)
    # i.e, {0: I_0, 1: I_1, 2: I_2, 3: I_3, 4: I_4, 5: I_5, 6: I_6, 7: I_7, 8: I_8, 9: I_9, 10: I_10, 11: I_11}

    # Inventory balance constraints
    for period, value in input_df_dict['input_data'].iloc[1:].iterrows():
        model.addConstraint(LpConstraint(
            e = inventory_variables[period - 1] + production_variables[period] - inventory_variables[period],
            sense = LpConstraintEQ,
            name ='inv_balance' + str(period),
            rhs = value.demand))
    #i.e,    1 I_0 - I_1 + X_1 = 3300
    #        2 I_1 - I_2 + X_2 = 4500
    #        3 I_2 - I_3 + X_3 = 6100
    #        4 I_3 - I_4 + X_4 = 5100
    #        5 I_4 - I_5 + X_5 = 9000
    #        6 I_5 - I_6 + X_6 = 9700
    #        7 I_6 - I_7 + X_7 = 8200
    #        8 I_7 - I_8 + X_8 = 7900
    #        9 I_8 - I_9 + X_9 = 5700
    #        10 -I_10 + I_9 + X_10 = 4500
    #        11 I_10 - I_11 + X_11 = 4200

    # Inventory balance for first period
    model.addConstraint(LpConstraint(
        e = production_variables[0] - inventory_variables[0],
        sense = LpConstraintEQ,
        name = 'inv_balance0',
        rhs = input_df_dict['input_data'].iloc[0].demand - input_param_dict['initial_inventory']))
    # i.e, (-1*I_0 + 1*X_0 + 0 = 3300 - 500)

    # Production capacity constraints
    for index, value in production_variables.items():
        model.addConstraint(LpConstraint(
            e = value,
            sense = LpConstraintLE,
            name = 'prod_cap_month_' + str(index),
            rhs = input_df_dict['input_data'].iloc[index].production_capacity))
    # i.e,   X_0 <= 6400
    #        X_1 <= 6400
    #        X_3 <= 6800
    #        X_4 <= 6400
    #        X_5 <= 7000
    #        X_6 <= 7000
    #        X_7 <= 7000
    #        X_8 <= 6800
    #        X_9 <= 6800
    #        X_10 <= 6800
    #        X_11 <= 6400
    
    # ----------------------------- additions ------------------------------------ #
    
#     # Production balance constarints
#     for period, value in input_df_dict['input_data'].iloc[0:].iterrows():
#         if period == 0:
#             model.addConstraint(LpConstraint(
#                 e=production_variables[period],
#                 sense=LpConstraintLE,
#                 name='prod_balance' + str(period),
#                 rhs=value.demand + (value.demand * input_param_dict['stock_threshold']) - input_param_dict['initial_inventory']))
#         else:
#             model.addConstraint(LpConstraint(
#                 e=inventory_variables[period - 1] + production_variables[period],
#                 sense=LpConstraintLE,
#                 name='prod_balance' + str(period),
#                 rhs=value.demand + (value.demand * input_param_dict['stock_threshold'])))

#     # Inventory capacity constraints
#     for index, value in inventory_variables.items():
#         model.addConstraint(LpConstraint(
#             e=value,
#             sense=LpConstraintLE,
#             name='inv_cap_month_' + str(index),
#             rhs=input_df_dict['input_data'].iloc[index].inventory_capacity))

    # ----------------------------- additions ------------------------------------ #

    # Costs and objective function
    total_holding_cost = input_param_dict['holding_cost'] * lpSum(inventory_variables)
    # i.e, 8*I_0 + 8*I_1 + 8*I_10 + 8*I_11 + 8*I_2 + 8*I_3 + 8*I_4 + 8*I_5 + 8*I_6 + 8*I_7 + 8*I_8 + 8*I_9 + 0

    total_production_cost = lpSum(row['production_cost'] * production_variables[index]
                                  for index, row in input_df_dict['input_data'].iterrows())
    # i.e, 150*X_0 + 150*X_1 + 140*X_10 + 140*X_11 + 140*X_2 + 140*X_3 + 150*X_4 + 135*X_5 + 135*X_6 + 135*X_7 + 135*X_8 + 140*X_9 + 0

    objective = total_holding_cost + total_production_cost

    model.setObjective(objective)

    model.solve()  
    print("Model Status = ", LpStatus[model.status])
    
    list_of_holding_costs = [input_param_dict['holding_cost'] for i in range(len(inventory_variables))]
    list_of_inventory_variables = list(inventory_variables.values())
    list_of_production_costs = input_df_dict['input_data'].production_cost
    list_of_production_variables = list(production_variables.values())
    
    df_holding_costs = pd.DataFrame({})
    df_production_costs = pd.DataFrame({})
    df_cost = pd.DataFrame({})
    
    for i in range(len(list_of_inventory_variables)):
        df_holding_costs = df_holding_costs.append({'Product': product, 'Inventory_Variable': list_of_inventory_variables[i], 
                                                    'Unit_Holding_Cost': list_of_holding_costs[i]}, ignore_index = True)
    for i in range(len(list_of_production_variables)):
        df_production_costs = df_production_costs.append({'Production_Variable': list_of_production_variables[i], 
                                                          'Unit_Production_Cost': list_of_production_costs[i]}, ignore_index = True)
    
    df_cost = pd.concat([df_holding_costs, df_production_costs], axis = 1, ignore_index = True)
    df_cost.columns = list(df_holding_costs.columns) + list(df_production_costs.columns)
    df_cost['Production_Capacity'] = input_df_dict['input_data']['production_capacity'].tolist()
#     df_cost['Inventory_Capacity'] = input_df_dict['input_data']['inventory_capacity'].tolist()
    df_cost['Demand'] = input_df_dict['input_data']['demand'].tolist()
    
    variableValueDict = {}
    variableValueDict = {v.name : v.varValue for v in model.variables()}

    for index, var in df_cost.iterrows():
        inv_var, prod_var = str(var.Inventory_Variable), str(var.Production_Variable)
        df_cost.loc[index, 'Inventory_Value'] = variableValueDict[inv_var]
        df_cost.loc[index, 'Production_Value'] = variableValueDict[prod_var]
        df_cost['Total_Cost'] = (df_cost['Inventory_Value'] * df_cost['Unit_Holding_Cost']) + (df_cost['Production_Value'] * df_cost['Unit_Production_Cost'])
    
    df_model_parameters = df_model_parameters.append({'Model_Name': model.name, 'Model_Objective': model.objective, 'Model_Status': LpStatus[model.status],
                                                      'Objective_Value': model.objective.value()}, ignore_index = True)
    df_costs = df_costs.append(df_cost, ignore_index=True)
    
#     # Each of the variables is printed with it's resolved optimum value
#     for v in model.variables():
#         print(v.name, "=", v.varValue)
    
    # The optimised objective function value is printed to the screen    
    print("Total Cost of Production = ", model.objective.value())
    print('************************************\n')

************************************
Product =  Coke
Model Status =  Optimal
Total Cost of Production =  10183400.0
************************************

************************************
Product =  Juice
Model Status =  Optimal
Total Cost of Production =  10712100.0
************************************



In [7]:
# df_costs.groupby('Product').sum()

In [8]:
df_costs

,Product,Inventory_Variable,Unit_Holding_Cost,Production_Variable,Unit_Production_Cost,Production_Capacity,Demand,Inventory_Value,Production_Value,Total_Cost
0,Coke,I_0,8.0,X_0,150.0,6400.0,3300.0,0.0,2800.0,420000.0
1,Coke,I_1,8.0,X_1,150.0,6400.0,3300.0,2700.0,6000.0,921600.0
2,Coke,I_2,8.0,X_2,140.0,6800.0,4500.0,5000.0,6800.0,992000.0
3,Coke,I_3,8.0,X_3,140.0,6800.0,6100.0,5700.0,6800.0,997600.0
4,Coke,I_4,8.0,X_4,150.0,6400.0,5100.0,7000.0,6400.0,1016000.0
5,Coke,I_5,8.0,X_5,135.0,7000.0,9000.0,5000.0,7000.0,985000.0
6,Coke,I_6,8.0,X_6,135.0,7000.0,9700.0,2300.0,7000.0,963400.0
7,Coke,I_7,8.0,X_7,135.0,7000.0,8200.0,1100.0,7000.0,953800.0
8,Coke,I_8,8.0,X_8,135.0,6800.0,7900.0,0.0,6800.0,918000.0
9,Coke,I_9,8.0,X_9,140.0,6800.0,5700.0,0.0,5700.0,798000.0


In [9]:
# path = "D:\Projects\Marico\InventoryOptimization\Extended Production Planning\Production_Plan_OP.xlsx"
# writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
# df_model_parameters.to_excel(writer, sheet_name = 'model_parameters', index = False)
# df_costs.to_excel(writer, sheet_name = 'variable_values', index = False)
# writer.save()
# writer.close()